### 1. 加载数据集

In [8]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [9]:
import sys
import os

# 添加项目根目录到Python路径
project_root = "/home/cuipeng/Gemma"
sys.path.append(project_root)

# 导入必要模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template

In [10]:
# config.py
from dotenv import load_dotenv # type: ignore

# 加载 .env 文件
load_dotenv()

True

In [11]:
from datasets import load_dataset # type: ignore

ceval_logic_dataset = load_dataset("ceval/ceval-exam", "logic", split="test")

In [12]:
ceval_logic_dataset, ceval_logic_dataset["question"]

(Dataset({
     features: ['id', 'question', 'A', 'B', 'C', 'D', 'answer', 'explanation'],
     num_rows: 204
 }),
 ['最近一项研究发现，海水颜色能够让飓风改变方向，也就是说，如果海水变色，飓风的移动路径也会变向。这也就意味着科学家可以根据海水的“脸色”判断哪些地方将被飓风袭击，哪些地区会幸免于难。值得关注的是，全球气候变暖可能已经让海水变色。以下哪项最可能是科学家作出判断所依赖的前提？____',
  '除非年龄在50岁以下，并且能持续游泳在三千米以上，否则不能参加下个月举行的横渡长江活动。同时，高血压和心脏病患者不能参加。老黄能持续游泳三千米以上，但没被批准参加这项活动。以上断定能推出以下哪项结论?____Ⅰ．老黄的年龄至少50岁。Ⅱ．老黄患有高血压。Ⅲ．老黄患有心脏病。',
  '有些人论证说：凡属中华人民共和国政府管辖的都是中国人，台湾人现在不受中华人民共和国政府管辖，所以，台湾人不是中国人。以下哪一个推理明显说明上述论证不成立?____',
  '纯种的蒙古奶牛一般每年产奶400升；如果蒙古奶牛与欧洲奶牛杂交，其后代一般每年可产2700升牛奶。为此，一个国际组织计划通过杂交的方式，帮助蒙古牧民提高其牛奶产量。以下哪项如果为真。对该国际组织的计划提出了最严重的质疑?____',
  '企业要建设科技创新中心，就要推进与高校、科研院所的合作，这样才能激发自主创新的活力。一个企业只有搭建服务科技创新发展战略的平台、科技创新与经济发展对接的平台以及聚集创新人才的平台，才能催生重大科技成果。根据上述信息,可以得出以下哪项?____',
  '张教授：如果在视觉上不能辨别艺术复制品和真品之间的差异，那么复制品就应该和真品的价值一样。因为如果两件艺术品在视觉上无差异，那么它们就有相同的品质。要是它们有相同的品质，它们的价格就应该相等。李研究员：你对艺术了解太少了!即使某人做了一件精致的复制品，并且在视觉上难以把这件复制品与真品区别开来，但由于这件复制品和真品产生于不同的年代，所以不能算有同样的品质。譬如说，现代人重塑的兵马俑再逼真，也不能与秦陵的兵马俑相提并论。以下哪项最恰当地指出上述张教授和李研究员的分歧所在?____',
  '班上同学的业余爱好多种多

### 2. 开始评估微调后的模型

In [ ]:
# 我们现在需要组合ceval_logic_dataset["question"]，ceval_logic_dataset["A"]，ceval_logic_dataset["B"]，ceval_logic_dataest["C"]，ceval_logic_dataest["D"]，生成一个完整的prompt让大模型进行回答

In [15]:
import os
from typing import List, Dict
from tqdm import tqdm # type: ignore

In [16]:
def create_ceval_logic_prompts(dataset) -> List[Dict[str, str]]:
    """
    为CEval逻辑数据集创建格式化的prompts
    
    Args:
        dataset: CEval逻辑数据集
        
    Returns:
        List[Dict[str, str]]: 包含格式化对话的列表
    """
    system_context = """你是一位逻辑推理专家。请仅用一个字母(A/B/C/D)回答问题,不需要解释。"""
    
    prompts = []
    
    for i in range(len(dataset)):
        question = dataset[i]["question"]
        option_a = dataset[i]["A"]
        option_b = dataset[i]["B"]
        option_c = dataset[i]["C"]
        option_d = dataset[i]["D"]
        
        user_prompt = f"""题目：{question}

A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d}

请直接回答选项字母。"""

        dialogue = [
            {"role": "system", "content": system_context},
            {"role": "user", "content": user_prompt}
        ]
        
        prompts.append(dialogue)
    
    return prompts

In [17]:
def process_ceval_logic_dataset(dataset, model, tokenizer):
    """
    处理整个CEval逻辑数据集
    
    Args:
        dataset: CEval逻辑数据集
        model: 已初始化的模型
        tokenizer: 已初始化的tokenizer
        
    Returns:
        dict: 包含处理结果的字典,格式为:
        {
            "logic": {
                "0": {"extracted": "A", "origin": "原始回答"},
                "1": {"extracted": "B", "origin": "原始回答"},
                ...
            }
        }
    """
    prompts = create_ceval_logic_prompts(dataset)
    results = {"logic": {}}
    
    # 添加进度条,设置描述信息
    for i, dialogue in tqdm(enumerate(prompts), total=len(prompts), desc="处理逻辑题目"):
        formatted_prompt = apply_chat_template(dialogue)
        response = generate_response(model, tokenizer, formatted_prompt)
        
        # 提取A/B/C/D答案
        extracted_answer = ''.join(char for char in response if char in 'ABCD')
        extracted_answer = extracted_answer[:1] if extracted_answer else ""
        
        # 保存结果
        results["logic"][str(i)] = {
            "extracted": extracted_answer,
            "origin": response.strip()
        }
    
    return results

In [18]:
# 使用示例:
model_path = "google/gemma-2-9b"
cache_dir = "/root/autodl-tmp/gemma"
lora_path = "/root/autodl-tmp/models/stage1/checkpoints/gemma-base-zh/checkpoint-43500"
model, tokenizer = initialize_model_and_tokenizer(
    model_path=model_path,
    cache_dir=cache_dir,
    lora_path=lora_path,
    use_quantization=True
)
results = process_ceval_logic_dataset(ceval_logic_dataset, model, tokenizer)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/root/miniconda3/envs/Gemma/lib/python3.12/site-packages/peft/tuners/adalora/config.py:78: UserWarning: Note that `r` is not used in AdaLora and will be ignored.If you intended to set the initial rank, use `init_r` instead.
  warnings.warn(
处理逻辑题目:   0%|          | 0/204 [00:00<?, ?it/s]/root/miniconda3/envs/Gemma/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.
处理逻辑题目: 100%|██████████| 204/204 [05:15<00:00,  1.55s/it]


In [19]:
import json

with open("eval_logic_lora.json", "w") as file:
    json.dump(results, file, indent=4)

### 3. 开始评估初始模型

In [20]:
# 使用示例:
model_path = "google/gemma-2-9b"
cache_dir = "/root/autodl-tmp/gemma"
lora_path = None
model, tokenizer = initialize_model_and_tokenizer(
    model_path=model_path,
    cache_dir=cache_dir,
    lora_path=lora_path,
    use_quantization=True
)
results = process_ceval_logic_dataset(ceval_logic_dataset, model, tokenizer)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

处理逻辑题目:   0%|          | 0/204 [00:00<?, ?it/s]/root/miniconda3/envs/Gemma/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
处理逻辑题目: 100%|██████████| 204/204 [44:38<00:00, 13.13s/it] 


In [21]:
import json

with open("eval_logic_base.json", "w") as file:
    json.dump(results, file, indent=4)

### 4. 处理得到的数据集使符合提交格式

In [22]:
def process_logic_data(input_json):
    """
    处理输入的逻辑题JSON数据，提取题号和答案
    
    Args:
        input_json (dict): 原始JSON数据
        
    Returns:
        dict: 处理后的数据，格式为 {"logic": {"0": "A", "1": "B", ...}}
    """
    result = {"logic": {}}
    
    # 获取logic部分的数据
    logic_data = input_json.get("logic", {})
    
    # 遍历所有题目
    for question_num, question_data in logic_data.items():
        # 获取extracted答案，如果没有则设为空字符串
        answer = question_data.get("extracted", "")
        # 将答案添加到结果中
        result["logic"][question_num] = answer
    
    return result

In [23]:
eval_logic_lora = json.load(open("eval_logic_lora.json", "r"))
eval_logic_base = json.load(open("eval_logic_base.json", "r"))

# 处理数据
processed_eval_logic_lora = process_logic_data(eval_logic_lora)
processed_eval_logic_base = process_logic_data(eval_logic_base)

# 将结果写入文件
with open('processed_eval_logic_lora.json', 'w', encoding='utf-8') as f:
    json.dump(processed_eval_logic_lora, f, ensure_ascii=False, indent=4)

with open('processed_eval_logic_base.json', 'w', encoding='utf-8') as f:
    json.dump(processed_eval_logic_base, f, ensure_ascii=False, indent=4)